In [82]:
import pandas as pd

In [83]:
# Set pandas to display all columns
pd.set_option('display.max_columns', None)

### Function that merges all months into one DF

In [84]:
def merge_csv(path, files):

    final_df = pd.DataFrame()
    for file in files:
        name = f'{path}{file}'
        print(name)
        df = pd.read_csv(name)
        final_df = pd.concat([final_df, df], ignore_index=True)

    return final_df

In [85]:

files = files = ['april_2024.csv', 'may_2024.csv', 'july_2024.csv', 'august_2024.csv', 'july_2024.csv', 'september_2024.csv', 'october_2024.csv']
#path = '/Users/rdhir/Documents/roni_dashboard_td2024/Data/' #Raghav Path
path = '/TAMUDATATHON24/roni_dashboard_td2024/Data/' #Travis Path
df = merge_csv(path, files)

/TAMUDATATHON24/roni_dashboard_td2024/Data/april_2024.csv
/TAMUDATATHON24/roni_dashboard_td2024/Data/may_2024.csv
/TAMUDATATHON24/roni_dashboard_td2024/Data/july_2024.csv
/TAMUDATATHON24/roni_dashboard_td2024/Data/august_2024.csv
/TAMUDATATHON24/roni_dashboard_td2024/Data/july_2024.csv
/TAMUDATATHON24/roni_dashboard_td2024/Data/september_2024.csv
/TAMUDATATHON24/roni_dashboard_td2024/Data/october_2024.csv


In [86]:
df.head()

,Order #,Sent Date,Modifier,Option Group Name,Parent Menu Selection,Order ID
0,18,2024-04-13 11:34:00,Regular,Noods,Mac and Cheese,13-18
1,18,2024-04-13 11:34:00,Cheddar,Choose Your Cheese,Mac and Cheese,13-18
2,18,2024-04-13 11:34:00,Pulled Pork,Choose Your Meats,Mac and Cheese,13-18
3,18,2024-04-13 11:34:00,Broccoli,Choose Your Toppings,Mac and Cheese,13-18
4,18,2024-04-13 11:34:00,Tomatoes,Choose Your Toppings,Mac and Cheese,13-18


### Add Column That Indicates New Dish Order

In [87]:
df["Dish End"] = (
    (df["Option Group Name"] == "Choose Your Drink") & 
    (df["Parent Menu Selection"].isin(["Mac and Cheese", "Grilled Cheese", "Drinks"]))
) | (df['Parent Menu Selection'].isin(['Sides/Desserts', 'Mac and Cheese Party Tray (Plus FREE Garlic Bread)',
       'Garlic Bread (Party Size)']))

In [88]:
# Initialize the dish counter and a list to hold the dish numbers
dish_counter = 1
dish_numbers = []

# Iterate over each row in `Dish End` and assign a dish number
for end in df["Dish End"]:
    dish_numbers.append(dish_counter)
    if end:  # If we encounter a `True`, increment the counter for the next sequence
        dish_counter += 1

# Add the dish numbers to the DataFrame
df["Dish Number"] = dish_numbers

# Create a unique `Dish ID` by combining `Order ID` and `Dish Number`
df["Dish ID"] = df["Order ID"] + "-" + df["Dish Number"].astype(str)

### Pivot DF so that row corresponds to a dish

In [89]:
df["Modifier"] = df["Modifier"].fillna("").astype(str)
df_pivoted = (
    df.groupby(["Dish ID", "Order ID", "Order #", "Dish Number", "Parent Menu Selection", "Sent Date", "Option Group Name"])["Modifier"]
    .apply(", ".join)  # Concatenate multiple selections in each option group
    .unstack(fill_value="")  # Pivot `Option Group Name` into columns, filling empty cells with an empty string
    .reset_index()  # Flatten the DataFrame for easier viewing
)

# Display the final result
df_pivoted.head(25)

Option Group Name,Dish ID,Order ID,Order #,Dish Number,Parent Menu Selection,Sent Date,Choose Your Cheese,Choose Your Drink,Choose Your Drizzles,Choose Your Meats,Choose Your Melted Cheese,Choose Your Shirt,Choose Your Side,Choose Your Toppings,Do you want Mac and Cheese added inside?,Mac and Cheese Options,Mix Bases,Noods,Sides,Utensils?
0,1-1-10108,1-1,1,10108,Mac and Cheese,2024-07-01 11:07:00,Cheddar,Unlimited Fountain Drinks,BBQ,Grilled Chicken,,,Cheesecake,"Broccoli, Corn, Onions, Tomatoes, Bell Peppers...",,,,Regular,,
1,1-1-12771,1-1,1,12771,Mac and Cheese,2024-08-01 10:41:00,Alfredo,No Drink,Ranch,Ham,,,Garlic Bread,Parmesan,,,,Regular,,
2,1-1-12772,1-1,1,12772,Mac and Cheese,2024-08-01 10:41:00,Pepper Jack,No Drink,BBQ,Pulled Pork,,,Garlic Bread,"Onions, Parmesan",,,,Regular,,
3,1-1-16802,1-1,1,16802,Mac and Cheese,2024-07-01 11:07:00,Cheddar,Unlimited Fountain Drinks,BBQ,Grilled Chicken,,,Cheesecake,"Broccoli, Corn, Onions, Tomatoes, Bell Peppers...",,,,Regular,,
4,1-1-19465,1-1,1,19465,Mac and Cheese,2024-09-01 11:08:00,Cheddar,Unlimited Fountain Drinks,Garlic Parmesan,Brisket,,,Cheesy Garlic Bread,"Broccoli, Breadcrumbs",,,,Regular,,
5,1-1-24157,1-1,1,24157,Mac and Cheese Party Tray (Plus FREE Garlic Br...,2024-10-01 10:42:00,,,,,,,,,,"Garden Mac (Cheddar, Broccoli, Tomatos, Ranch)",,,,
6,1-1-24158,1-1,1,24158,Mac and Cheese Party Tray (Plus FREE Garlic Br...,2024-10-01 10:42:00,,,,,,,,,,"Chicken Alfredo (Alfredo, Chicken, Broccoli, P...",,,,
7,1-1-24159,1-1,1,24159,Mac and Cheese Party Tray (Plus FREE Garlic Br...,2024-10-01 10:42:00,,,,,,,,,,"Texas BBQ (Pepper Jack, Brisket, Jalapenos, BBQ)",,,,
8,1-1-24160,1-1,1,24160,Mac and Cheese Party Tray (Plus FREE Garlic Br...,2024-10-01 10:42:00,,,,,,,,,,"Chicken Bacon Ranch (Cheddar, Chicken, Bacon, ...",,,,
9,1-1-24161,1-1,1,24161,Mac and Cheese Party Tray (Plus FREE Garlic Br...,2024-10-01 10:42:00,,,,,,,,,,"Buffalo Chicken (Cheddar, Chicken, Onions, Buf...",,,,


### Combine Mix Cheese rows with actual dish

In [90]:
# Step 1: Identify rows with "MIX" in `Parent Menu Selection`
mix_rows = df_pivoted[df_pivoted["Parent Menu Selection"] == "MIX"]

# Step 2: For each "MIX" row, combine its `Mix Bases` with `Choose Your Cheese` in the actual dish row
for index, mix_row in mix_rows.iterrows():
    dish_id = mix_row["Dish ID"]
    actual_dish_row = df_pivoted[(df_pivoted["Dish ID"] == dish_id) & (df_pivoted["Parent Menu Selection"] != "MIX")]

    if not actual_dish_row.empty:
        # Get the combined cheese list from `Mix Bases` and the existing `Choose Your Cheese`
        combined_cheese = f"{mix_row['Mix Bases']}, {actual_dish_row.iloc[0]['Choose Your Cheese']}".strip(", ")
        
        # Update the `Choose Your Cheese` column in the actual dish row
        df_pivoted.loc[actual_dish_row.index, "Choose Your Cheese"] = combined_cheese

# Step 3: Drop the "MIX" rows
df_pivoted = df_pivoted[df_pivoted["Parent Menu Selection"] != "MIX"]

# Display the result
df_pivoted.reset_index(drop=True, inplace=True)


### Add Columns Month, Day of Week

In [91]:
def add_date_time_columns(df, date_column):
    # Ensure the date column is in datetime format
    df[date_column] = pd.to_datetime(df[date_column], errors='coerce')
    
    # Extract month name
    df['Month'] = df[date_column].dt.month_name()
    
    # Extract day of the week
    df['Day of the Week'] = df[date_column].dt.day_name()
    
    # Extract time (in HH:MM:SS format)
    df['Time'] = df[date_column].dt.time
    
    return df

In [92]:
df = add_date_time_columns(df_pivoted, 'Sent Date').drop(columns = ['Order ID', 'Dish Number'])

In [93]:
df

Option Group Name,Dish ID,Order #,Parent Menu Selection,Sent Date,Choose Your Cheese,Choose Your Drink,Choose Your Drizzles,Choose Your Meats,Choose Your Melted Cheese,Choose Your Shirt,Choose Your Side,Choose Your Toppings,Do you want Mac and Cheese added inside?,Mac and Cheese Options,Mix Bases,Noods,Sides,Utensils?,Month,Day of the Week,Time
0,1-1-10108,1,Mac and Cheese,2024-07-01 11:07:00,Cheddar,Unlimited Fountain Drinks,BBQ,Grilled Chicken,,,Cheesecake,"Broccoli, Corn, Onions, Tomatoes, Bell Peppers...",,,,Regular,,,July,Monday,11:07:00
1,1-1-12771,1,Mac and Cheese,2024-08-01 10:41:00,Alfredo,No Drink,Ranch,Ham,,,Garlic Bread,Parmesan,,,,Regular,,,August,Thursday,10:41:00
2,1-1-12772,1,Mac and Cheese,2024-08-01 10:41:00,Pepper Jack,No Drink,BBQ,Pulled Pork,,,Garlic Bread,"Onions, Parmesan",,,,Regular,,,August,Thursday,10:41:00
3,1-1-16802,1,Mac and Cheese,2024-07-01 11:07:00,Cheddar,Unlimited Fountain Drinks,BBQ,Grilled Chicken,,,Cheesecake,"Broccoli, Corn, Onions, Tomatoes, Bell Peppers...",,,,Regular,,,July,Monday,11:07:00
4,1-1-19465,1,Mac and Cheese,2024-09-01 11:08:00,Cheddar,Unlimited Fountain Drinks,Garlic Parmesan,Brisket,,,Cheesy Garlic Bread,"Broccoli, Breadcrumbs",,,,Regular,,,September,Sunday,11:08:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31771,9-98-25198,98,Mac and Cheese,2024-10-09 19:18:00,Cheddar,No Drink,"Buffalo, Ranch",Grilled Chicken,,,No Side,"Bell Peppers, Breadcrumbs",,,,Regular,,,October,Wednesday,19:18:00
31772,9-98-8125,98,Mac and Cheese,2024-05-09 16:03:00,Cheddar,Barq's Root Beer,Ranch,Bacon,,,Cheesecake,Pineapple,,,,Regular,,,May,Thursday,16:03:00
31773,9-99-20863,99,Mac and Cheese,2024-09-09 16:13:00,"Cheddar, Pepper Jack, Alfredo, MIX",No Drink,Garlic Parmesan,No Meat,,,No Side,"Corn, Parmesan, Breadcrumbs",,,,Regular,,,September,Monday,16:13:00
31774,9-99-25196,99,Mac and Cheese,2024-10-09 19:18:00,"Cheddar, Pepper Jack, MIX",No Drink,"Buffalo, Ranch","Pulled Pork, Bacon",,,No Side,"Onions, Breadcrumbs",,,,Regular,,,October,Wednesday,19:18:00


In [94]:
df_pivoted['Choose Your Drizzles'].unique()

array(['BBQ', 'Ranch', 'Garlic Parmesan', '', 'No Drizzle', 'Hot Honey',
       'Pesto', 'Hot Honey, Buffalo', 'BBQ, Ranch',
       'Garlic Parmesan, Pesto', 'Buffalo',
       'Garlic Parmesan, Pesto, Hot Honey', 'Buffalo, Ranch',
       'BBQ, Buffalo, Pesto, Hot Honey', 'Garlic Parmesan, Pesto, Ranch',
       'BBQ, Hot Honey', 'Garlic Parmesan, Ranch',
       'Garlic Parmesan, Ranch, Hot Honey', 'BBQ, Buffalo',
       'BBQ, Garlic Parmesan', 'Garlic Parmesan, Pesto, Ranch, Hot Honey',
       'No Drizzle, BBQ', 'Garlic Parmesan, Buffalo',
       'Buffalo, Hot Honey', 'Ranch, BBQ', 'Pesto, Ranch', 'BBQ, Pesto',
       'No Drizzle, Ranch', 'No Drizzle, Garlic Parmesan',
       'BBQ, Garlic Parmesan, Hot Honey',
       'Garlic Parmesan, Buffalo, Ranch',
       'Hot Honey, BBQ, Garlic Parmesan', 'BBQ, Garlic Parmesan, Buffalo',
       'Garlic Parmesan, Buffalo, Ranch, Hot Honey',
       'Garlic Parmesan, Hot Honey', 'Ranch, Hot Honey',
       'Garlic Parmesan, Buffalo, Pesto, Ranch', 'BBQ,

In [95]:
# Helper function to create dummy variables based on the desired list
def create_dummies(df, column, desired_values):
    # Split the values in each cell by commas, expand into dummy variables
    dummies = df[column].str.get_dummies(sep=', ')
    
    # Filter to keep only the desired columns
    dummies = dummies[desired_values]
    
    # Rename columns to indicate original column source
    dummies.columns = [f"{val}" for val in dummies.columns]
    
    return dummies

In [96]:
desired_cheeses = ["Cheddar", "Pepper Jack", "Alfredo"]
cheese_dummies = create_dummies(df_pivoted, "Choose Your Cheese", desired_cheeses)

desired_meats = ["Grilled Chicken", "Pulled Pork", "Brisket", "No Meat", "Ham", "Bacon"]
meat_dummies = create_dummies(df, "Choose Your Meats", desired_meats)

desired_drizzles = ['BBQ', 'Ranch', 'Garlic Parmesan', 'No Drizzle', 'Hot Honey', 'Buffalo']
drizzle_dummies = create_dummies(df, "Choose Your Drizzles", desired_drizzles)

sides = ['Cheesy Garlic Bread', 'No Side', 'Garlic Bread', 'Cheesecake', 'Doritos', 'Cheetos', 'Lays Barbecue', 'Side Mac', 'Large Chocolate Chunk Cookie', 'Cheesy Broccoli', 'Lays Classic']
sides_dummies = create_dummies(df, "Choose Your Side", sides)


In [97]:
df_with_dummies = pd.concat([df, cheese_dummies, meat_dummies, drizzle_dummies, sides_dummies], axis=1)

In [98]:
df_with_dummies.head(50)

,Dish ID,Order #,Parent Menu Selection,Sent Date,Choose Your Cheese,Choose Your Drink,Choose Your Drizzles,Choose Your Meats,Choose Your Melted Cheese,Choose Your Shirt,Choose Your Side,Choose Your Toppings,Do you want Mac and Cheese added inside?,Mac and Cheese Options,Mix Bases,Noods,Sides,Utensils?,Month,Day of the Week,Time,Cheddar,Pepper Jack,Alfredo,Grilled Chicken,Pulled Pork,Brisket,No Meat,Ham,Bacon,BBQ,Ranch,Garlic Parmesan,No Drizzle,Hot Honey,Buffalo,Cheesy Garlic Bread,No Side,Garlic Bread,Cheesecake,Doritos,Cheetos,Lays Barbecue,Side Mac,Large Chocolate Chunk Cookie,Cheesy Broccoli,Lays Classic
0,1-1-10108,1,Mac and Cheese,2024-07-01 11:07:00,Cheddar,Unlimited Fountain Drinks,BBQ,Grilled Chicken,,,Cheesecake,"Broccoli, Corn, Onions, Tomatoes, Bell Peppers...",,,,Regular,,,July,Monday,11:07:00,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,1-1-12771,1,Mac and Cheese,2024-08-01 10:41:00,Alfredo,No Drink,Ranch,Ham,,,Garlic Bread,Parmesan,,,,Regular,,,August,Thursday,10:41:00,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,1-1-12772,1,Mac and Cheese,2024-08-01 10:41:00,Pepper Jack,No Drink,BBQ,Pulled Pork,,,Garlic Bread,"Onions, Parmesan",,,,Regular,,,August,Thursday,10:41:00,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,1-1-16802,1,Mac and Cheese,2024-07-01 11:07:00,Cheddar,Unlimited Fountain Drinks,BBQ,Grilled Chicken,,,Cheesecake,"Broccoli, Corn, Onions, Tomatoes, Bell Peppers...",,,,Regular,,,July,Monday,11:07:00,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,1-1-19465,1,Mac and Cheese,2024-09-01 11:08:00,Cheddar,Unlimited Fountain Drinks,Garlic Parmesan,Brisket,,,Cheesy Garlic Bread,"Broccoli, Breadcrumbs",,,,Regular,,,September,Sunday,11:08:00,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
5,1-1-24157,1,Mac and Cheese Party Tray (Plus FREE Garlic Br...,2024-10-01 10:42:00,,,,,,,,,,"Garden Mac (Cheddar, Broccoli, Tomatos, Ranch)",,,,,October,Tuesday,10:42:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1-1-24158,1,Mac and Cheese Party Tray (Plus FREE Garlic Br...,2024-10-01 10:42:00,,,,,,,,,,"Chicken Alfredo (Alfredo, Chicken, Broccoli, P...",,,,,October,Tuesday,10:42:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,1-1-24159,1,Mac and Cheese Party Tray (Plus FREE Garlic Br...,2024-10-01 10:42:00,,,,,,,,,,"Texas BBQ (Pepper Jack, Brisket, Jalapenos, BBQ)",,,,,October,Tuesday,10:42:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,1-1-24160,1,Mac and Cheese Party Tray (Plus FREE Garlic Br...,2024-10-01 10:42:00,,,,,,,,,,"Chicken Bacon Ranch (Cheddar, Chicken, Bacon, ...",,,,,October,Tuesday,10:42:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1-1-24161,1,Mac and Cheese Party Tray (Plus FREE Garlic Br...,2024-10-01 10:42:00,,,,,,,,,,"Buffalo Chicken (Cheddar, Chicken, Onions, Buf...",,,,,October,Tuesday,10:42:00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [99]:
df['Choose Your Side'].unique()

array(['Cheesecake', 'Garlic Bread', 'Cheesy Garlic Bread', '', 'No Side',
       'No Side, Cheesy Garlic Bread', 'Large Chocolate Chunk Cookie',
       'No Side, No Side', 'Any Bag Of Chips', 'Side Mac',
       'Lays Barbecue', 'Cheesy Garlic Bread, Side Mac',
       'Cheesy Garlic Bread, Cheesecake', 'No Side, Any Bag Of Chips',
       'No Side, Garlic Bread', 'Cheesy Garlic Bread, Side Mac, Doritos',
       'No Side, Cheesecake',
       'Any Bag Of Chips, Large Chocolate Chunk Cookie', 'Cheetos',
       'Doritos', 'Garlic Bread, Doritos, Lays Classic',
       'Garlic Bread, Cheesy Garlic Bread, Cheesecake',
       'No Side, Doritos',
       'Cheesy Garlic Bread, Large Chocolate Chunk Cookie',
       'Garlic Bread, Cheesy Garlic Bread', 'Cheesecake, No Side',
       'Garlic Bread, Side Mac', 'Cheesy Garlic Bread, Any Bag Of Chips',
       'Garlic Bread, Cheesecake',
       'No Side, Cheesy Garlic Bread, Any Bag Of Chips',
       'No Side, Large Chocolate Chunk Cookie', 'Side Mac, Gar

In [100]:
df_with_dummies.describe()

,Order #,Sent Date,Cheddar,Pepper Jack,Alfredo,Grilled Chicken,Pulled Pork,Brisket,No Meat,Ham,Bacon,BBQ,Ranch,Garlic Parmesan,No Drizzle,Hot Honey,Buffalo,Cheesy Garlic Bread,No Side,Garlic Bread,Cheesecake,Doritos,Cheetos,Lays Barbecue,Side Mac,Large Chocolate Chunk Cookie,Cheesy Broccoli,Lays Classic
count,31776.000000,31776,31776.000000,31776.000000,31776.000000,31776.000000,31776.000000,31776.000000,31776.000000,31776.000000,31776.000000,31776.000000,31776.000000,31776.000000,31776.000000,31776.000000,31776.000000,31776.000000,31776.000000,31776.000000,31776.000000,31776.000000,31776.000000,31776.000000,31776.000000,31776.000000,31776.000000,31776.000000
mean,110.854828,2024-07-18 14:31:11.722054400,0.579494,0.203172,0.180167,0.271935,0.087047,0.299597,0.156219,0.034428,0.202448,0.217145,0.161506,0.293681,0.176737,0.096677,0.124685,0.230079,0.561052,0.089029,0.038866,0.002455,0.001794,0.001259,0.011738,0.004972,0.000094,0.001511
min,1.000000,2024-04-13 11:34:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,35.000000,2024-05-06 16:54:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,72.000000,2024-07-24 12:09:00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,139.000000,2024-09-12 11:51:30,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,7002.000000,2024-10-31 21:46:00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
std,183.644123,NaN,0.493648,0.402366,0.384333,0.444963,0.281908,0.458089,0.363068,0.182330,0.401831,0.412309,0.368002,0.455455,0.381452,0.295522,0.330367,0.420890,0.496266,0.284791,0.193278,0.049485,0.042316,0.035458,0.107708,0.070340,0.009716,0.038837


In [102]:
df_with_dummies.to_csv('ronisCleaned.csv', index=False)